In [1]:
from PIL import Image
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt

DIR = 'C:\Yelp\Train'

# Want to know how we should format the height x width image data dimensions
# for inputting to a keras model
def get_size_statistics():
    heights = []
    widths = []
    img_count = 0
    for img in os.listdir(DIR):
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            data = np.array(Image.open(path))
            heights.append(data.shape[0])
            widths.append(data.shape[1])
            img_count += 1
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    print("Average Height: " + str(avg_height))
    print("Max Height: " + str(max(heights)))
    print("Min Height: " + str(min(heights)))
    print('\n')
    print("Average Width: " + str(avg_width))
    print("Max Width: " + str(max(widths)))
    print("Min Width: " + str(min(widths)))
get_size_statistics()

Average Height: 383.958217270195
Max Height: 400
Min Height: 150


Average Width: 482.1309192200557
Max Width: 600
Min Width: 200


In [3]:
def label_img(name):
    word_label = name.split('(')[0]
    #print(word_label)
    if word_label == 'Burger ':  return 0
    elif word_label == 'pizza ': return 1
    elif word_label == 'Chicken ': return 2
    elif word_label == 'Sweet ': return 3
    elif word_label == 'Tacos ': return 4
    elif word_label == 'Drink ': return 5

In [4]:
IMG_SIZE = 256
def load_training_data():
    train_data = []
    for img in os.listdir(DIR):
        label = label_img(img)
        path = os.path.join(DIR, img)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            train_data.append([np.array(img), label])
            X_train=np.array(img)
            Y_train=label
    shuffle(train_data)
    return train_data

In [5]:
train_data = np.array(load_training_data())
y=len(train_data)
X=[]
Y=[]

In [6]:
for i in range(y-1):
    img=np.array((train_data[i][0]).reshape(256,256))
    X.append(img)
    label=np.array(train_data[i][1])
    Y.append(label)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    random_state=42,
                                                    test_size=0.20)

In [8]:
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(286, 256, 256)
(72, 256, 256)
(286,)
(72,)


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
from keras.utils import to_categorical
import numpy as np

Using TensorFlow backend.


In [30]:
classifier=Sequential()
classifier.add(Conv2D(64,3,3,input_shape=(256,256,1),activation='relu'))
classifier.add(Conv2D(32,3,3,activation='relu'))
#classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(output_dim=6,activation='softmax'))
classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

D:\Python\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(256, 256,..., activation="relu")`
  
D:\Python\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Python\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=6)`
  


In [31]:
X_train = X_train.reshape(286,256,256,1)
X_test = X_test.reshape(72,256,256,1)
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [32]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 254, 254, 64)      640       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 252, 252, 32)      18464     
_________________________________________________________________
flatten_3 (Flatten)          (None, 2032128)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 12192774  
Total params: 12,211,878
Trainable params: 12,211,878
Non-trainable params: 0
_________________________________________________________________


In [33]:
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 286 samples, validate on 72 samples
Epoch 1/3
286/286 [==============================] - 99s 345ms/step - loss: 11.8148 - acc: 0.2657 - val_loss: 12.3124 - val_acc: 0.2361
Epoch 2/3
286/286 [==============================] - 96s 337ms/step - loss: 11.4405 - acc: 0.2902 - val_loss: 12.3124 - val_acc: 0.2361
Epoch 3/3
286/286 [==============================] - 101s 352ms/step - loss: 11.4405 - acc: 0.2902 - val_loss: 12.3124 - val_acc: 0.2361


In [34]:
classifier.evaluate(X_test, y_test) 

72/72 [==============================] - 5s 72ms/step


[12.312433560689291, 0.2361111111111111]

In [35]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(256,256,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(6, activation = 'softmax'))

In [36]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 254, 254, 32)      320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 127, 127, 32)      128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 60, 60, 96)        55392     
__________

In [37]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics = ['accuracy'])

In [38]:
model.fit(X_train,y_train,epochs = 10, validation_steps=None,verbose = 1,batch_size=50)

Epoch 1/10
286/286 [==============================] - 40s 140ms/step - loss: 1.7728 - acc: 0.2692
Epoch 2/10
286/286 [==============================] - 43s 149ms/step - loss: 1.4387 - acc: 0.4650
Epoch 3/10
286/286 [==============================] - 45s 158ms/step - loss: 1.3696 - acc: 0.5315
Epoch 4/10
286/286 [==============================] - 43s 150ms/step - loss: 1.1996 - acc: 0.5524
Epoch 5/10
286/286 [==============================] - 43s 149ms/step - loss: 1.0978 - acc: 0.6224
Epoch 6/10
286/286 [==============================] - 41s 144ms/step - loss: 0.9595 - acc: 0.6888
Epoch 7/10
286/286 [==============================] - 40s 139ms/step - loss: 0.9259 - acc: 0.7133
Epoch 8/10
286/286 [==============================] - 40s 139ms/step - loss: 0.8669 - acc: 0.7203
Epoch 9/10
286/286 [==============================] - 40s 139ms/step - loss: 0.7832 - acc: 0.7622
Epoch 10/10
286/286 [==============================] - 40s 140ms/step - loss: 0.7264 - acc: 0.7657


In [39]:
model.evaluate(X_test, y_test)

72/72 [==============================] - 3s 38ms/step


[1.391570806503296, 0.5416666666666666]

In [40]:
loss,acc = model.evaluate(X_test, y_test, verbose = 0)
print(acc * 100)

54.166666666666664


In [42]:
predictions=model.predict(X_test)

In [44]:
Pred_Max_Positions=[]
for i in range(72):
    res=max(predictions[i])
    #print(res)
    for j in range(2):
        if (res==predictions[i][j]):
            result=j+1
            Pred_Max_Positions.append(j)
#Pred_Max_Positions

In [46]:
Test_Max_Positions=[]
for i in range(72):
    res=max(y_test[i])
    #print(res)
    for j in range(2):
        if (res==y_test[i][j]):
            result=j
            Test_Max_Positions.append(j)
#Test_Max_Positions


In [49]:
len(Pred_Max_Positions)

55